# Diabetes or not: A Comprehensive Analysis of Diabetical Symptoms in Pima Indians Diabetes Database #

### by Brian Adhitya, Wyatt Howell, Brian Ko, Nishanth Selvakumar

### Introduction

### Loading libraries

### Preeliminary exploratory data analysis

**1. Loading the Dataset**

**2. Cleaning columns and rows**

**3. Splitting into testing and training**

**4. Cleaning and wrangling data**

**5. Visualizing some key points**

### Methods

### Expected Outcomes and Significance

### Bibliography